In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import model_selection
import matplotlib.pyplot as plt
import torch
import scipy.stats as stats
import torch.nn as nn

In [3]:
X_numpy = np.load('lasso.npy', allow_pickle=True)
y_numpy = np.load('label.npy', allow_pickle=True)

In [4]:
X = torch.from_numpy(X_numpy.astype(np.float32)).squeeze()
y = torch.from_numpy(y_numpy.astype(np.float32))

In [5]:
#lets split our data
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2,
                                                                           random_state= 51)

In [6]:
#send the data into gpu
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
X_train,X_test, y_train,y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

In [8]:
# lets create a model for hyperparameter tuning
class NeuralNetworkWithHyper(nn.Module):
    def __init__(self,hidden_layers, num_hidden_units, input_size=10):
        super().__init__()
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.num_hidden_units = num_hidden_units
        
        # Create a list to hold the layers
        self.layers = nn.ModuleList()
        
        #add the input layer
        self.layers.append(nn.Linear(input_size, num_hidden_units))
        self.layers.append(nn.ReLU())
       
        #add hidden layers
        for i in range(hidden_layers - 1):
            self.layers.append(nn.Linear(num_hidden_units, num_hidden_units))
            self.layers.append(nn.ReLU())
        #add the output layer
        self.layers.append(nn.Linear(num_hidden_units, 1))
        
        #add forward function
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [23]:
#Lets set our loss function and optimizer
import optuna

def objective(trial):
    
    #define the search space
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3)
    hidden_layers = trial.suggest_int("hidden_layers", 1, 1)
    num_hidden_units = trial.suggest_int("num_hidden_units", 16, 46)
    epoch_count = trial.suggest_int("epoch_count", 9000, 11000)
    #weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-3)
    
    #create the model
    model = NeuralNetworkWithHyper(hidden_layers, 
                                   num_hidden_units 
                                   ).to('mps')
    optimizer = torch.optim.SGD(model.parameters(), learning_rate, #weight_decay
                               )
    loss_fn = torch.nn.L1Loss()
    
    #testing loop
    for epoch in range(epoch_count):
        #convert into train
        model.train()

        #fit data
        train_pred = model(X_train)

        #calculate loss
        train_loss = loss_fn(train_pred, y_train)


        #zero gradients
        optimizer.zero_grad()

        #back propagation
        train_loss.backward()

        #gradient descent
        optimizer.step()

        #testing
        model.eval()
        with torch.inference_mode():
            test_pred = model(X_test)
            test_loss = loss_fn(test_pred, y_test)
            spear_pred = test_pred.to('cpu')
            
    #spearman correlation coefficient       
    validation_metric = stats.spearmanr(y_test.to('cpu'), spear_pred).statistic
    
    #save model weights
    trial.set_user_attr('model_weights', model.state_dict())
    
    return validation_metric

#Create an Optuna Study
study = optuna.create_study(direction='maximize')

#Perform hyperparameter optimization
study.optimize(objective, n_trials=200)

#Print best spearman correlation coefficient
best_validation_metric = study.best_value

[I 2023-11-12 11:35:41,966] A new study created in memory with name: no-name-04a3edce-d948-4b3e-a015-d979941e81b3
[I 2023-11-12 11:36:03,369] Trial 0 finished with value: 0.25709750927400976 and parameters: {'learning_rate': 0.0007292879066943576, 'hidden_layers': 1, 'num_hidden_units': 44, 'epoch_count': 9369}. Best is trial 0 with value: 0.25709750927400976.
[I 2023-11-12 11:36:22,771] Trial 1 finished with value: 0.19666295854465735 and parameters: {'learning_rate': 0.0005634383458498917, 'hidden_layers': 1, 'num_hidden_units': 36, 'epoch_count': 9419}. Best is trial 0 with value: 0.25709750927400976.
[I 2023-11-12 11:36:45,422] Trial 2 finished with value: 0.20743828848238066 and parameters: {'learning_rate': 0.0003999609517790664, 'hidden_layers': 1, 'num_hidden_units': 29, 'epoch_count': 10604}. Best is trial 0 with value: 0.25709750927400976.
[I 2023-11-12 11:37:08,011] Trial 3 finished with value: 0.2334945873087682 and parameters: {'learning_rate': 0.0005301546720119393, 'hidd

[I 2023-11-12 11:47:10,198] Trial 33 finished with value: 0.2808036840623187 and parameters: {'learning_rate': 0.0009333745179179499, 'hidden_layers': 1, 'num_hidden_units': 29, 'epoch_count': 9648}. Best is trial 30 with value: 0.2995092797369395.
[I 2023-11-12 11:47:29,299] Trial 34 finished with value: 0.2559538720272084 and parameters: {'learning_rate': 0.0009230749062175324, 'hidden_layers': 1, 'num_hidden_units': 29, 'epoch_count': 9452}. Best is trial 30 with value: 0.2995092797369395.
[I 2023-11-12 11:47:48,758] Trial 35 finished with value: 0.2099767366912279 and parameters: {'learning_rate': 0.0009263313787389543, 'hidden_layers': 1, 'num_hidden_units': 27, 'epoch_count': 9636}. Best is trial 30 with value: 0.2995092797369395.
[I 2023-11-12 11:48:08,111] Trial 36 finished with value: 0.22269232185198382 and parameters: {'learning_rate': 0.0008448006198031441, 'hidden_layers': 1, 'num_hidden_units': 33, 'epoch_count': 9389}. Best is trial 30 with value: 0.2995092797369395.
[I 

[I 2023-11-12 11:58:01,880] Trial 66 finished with value: 0.2625827032681384 and parameters: {'learning_rate': 0.0008670678836160764, 'hidden_layers': 1, 'num_hidden_units': 33, 'epoch_count': 9528}. Best is trial 30 with value: 0.2995092797369395.
[I 2023-11-12 11:58:21,406] Trial 67 finished with value: 0.2480999235943172 and parameters: {'learning_rate': 0.0008273094138904905, 'hidden_layers': 1, 'num_hidden_units': 30, 'epoch_count': 9581}. Best is trial 30 with value: 0.2995092797369395.
[I 2023-11-12 12:00:10,173] Trial 68 finished with value: 0.24931763351853656 and parameters: {'learning_rate': 0.0007732530472742418, 'hidden_layers': 1, 'num_hidden_units': 32, 'epoch_count': 9818}. Best is trial 30 with value: 0.2995092797369395.
[I 2023-11-12 12:01:26,835] Trial 69 finished with value: 0.2730332357385169 and parameters: {'learning_rate': 0.0008556783006699623, 'hidden_layers': 1, 'num_hidden_units': 30, 'epoch_count': 9703}. Best is trial 30 with value: 0.2995092797369395.
[I 

[I 2023-11-12 13:44:47,458] Trial 99 finished with value: 0.26240425545435875 and parameters: {'learning_rate': 0.000935237207202856, 'hidden_layers': 1, 'num_hidden_units': 42, 'epoch_count': 9945}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 13:45:07,087] Trial 100 finished with value: 0.27720779226766545 and parameters: {'learning_rate': 0.000899691539523804, 'hidden_layers': 1, 'num_hidden_units': 36, 'epoch_count': 10007}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 13:45:26,884] Trial 101 finished with value: 0.23759147975795739 and parameters: {'learning_rate': 0.000861705354682434, 'hidden_layers': 1, 'num_hidden_units': 38, 'epoch_count': 10044}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 13:45:48,520] Trial 102 finished with value: 0.29815352527753214 and parameters: {'learning_rate': 0.0009200428540008654, 'hidden_layers': 1, 'num_hidden_units': 36, 'epoch_count': 10141}. Best is trial 87 with value: 0.333216612752

[I 2023-11-12 16:16:39,341] Trial 132 finished with value: 0.22586824401194314 and parameters: {'learning_rate': 0.0009194254009397327, 'hidden_layers': 1, 'num_hidden_units': 34, 'epoch_count': 10079}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:16:59,225] Trial 133 finished with value: 0.22370891323388048 and parameters: {'learning_rate': 0.0008385740598064058, 'hidden_layers': 1, 'num_hidden_units': 32, 'epoch_count': 10223}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:17:18,812] Trial 134 finished with value: 0.2692485708477773 and parameters: {'learning_rate': 0.0008778499082464941, 'hidden_layers': 1, 'num_hidden_units': 36, 'epoch_count': 10043}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:17:37,555] Trial 135 finished with value: 0.24233482466461226 and parameters: {'learning_rate': 0.0009000662828592914, 'hidden_layers': 1, 'num_hidden_units': 31, 'epoch_count': 9546}. Best is trial 87 with value: 0.333216612

[I 2023-11-12 16:27:18,816] Trial 165 finished with value: 0.3302144246906094 and parameters: {'learning_rate': 0.0008984053406016353, 'hidden_layers': 1, 'num_hidden_units': 38, 'epoch_count': 9764}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:27:38,026] Trial 166 finished with value: 0.2504873084337636 and parameters: {'learning_rate': 0.0008993176315929023, 'hidden_layers': 1, 'num_hidden_units': 38, 'epoch_count': 9807}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:27:57,199] Trial 167 finished with value: 0.22537689525173743 and parameters: {'learning_rate': 0.0009239140304739327, 'hidden_layers': 1, 'num_hidden_units': 34, 'epoch_count': 9745}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:28:16,040] Trial 168 finished with value: 0.2891034153356706 and parameters: {'learning_rate': 0.000976193713554297, 'hidden_layers': 1, 'num_hidden_units': 33, 'epoch_count': 9621}. Best is trial 87 with value: 0.333216612752624

[I 2023-11-12 16:38:09,343] Trial 198 finished with value: 0.2332232119542279 and parameters: {'learning_rate': 0.0009123583590200399, 'hidden_layers': 1, 'num_hidden_units': 34, 'epoch_count': 9569}. Best is trial 87 with value: 0.33321661275262476.
[I 2023-11-12 16:38:29,175] Trial 199 finished with value: 0.2544185474407272 and parameters: {'learning_rate': 0.0007604353128727908, 'hidden_layers': 1, 'num_hidden_units': 36, 'epoch_count': 10022}. Best is trial 87 with value: 0.33321661275262476.


### Trial #1

In [15]:
best_trial = study.best_trial
best_params = best_trial.params
best_score = best_trial.value
print(f'Best Score: {best_score} \nHyperparameters: ')
for key,value in best_params.items():
    print(f'{key}: {value}')
trials.update({"lasso_1": best_score})

Best Score: 0.31736281515859505 
Hyperparameters: 
learning_rate: 0.000497262490910934
hidden_layers: 1
num_hidden_units: 40
epoch_count: 11600
weight_decay: 0.0007379496783911263


### Trial #2

In [17]:
best_trial = study.best_trial
best_params = best_trial.params
best_score = best_trial.value
print(f'Best Score: {best_score} \nHyperparameters: ')
for key,value in best_params.items():
    print(f'{key}: {value}')

Best Score: 0.31565824572734685 
Hyperparameters: 
learning_rate: 0.00024505307860092456
hidden_layers: 1
num_hidden_units: 33
epoch_count: 9766
weight_decay: 0.0006367294579189647


### Trial #3

In [24]:
best_trial = study.best_trial
best_params = best_trial.params
best_score = best_trial.value
print(f'Best Score: {best_score} \nHyperparameters: ')
for key,value in best_params.items():
    print(f'{key}: {value}')

Best Score: 0.33321661275262476 
Hyperparameters: 
learning_rate: 0.0007545975231958816
hidden_layers: 1
num_hidden_units: 37
epoch_count: 10094
